In [105]:
import numpy as np
import Amazon_Movie_Parser as amp
# save numpy array as csv file
from numpy import savetxt
# load numpy array from csv file
from numpy import loadtxt

In [86]:
# Normalization test
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])
#print(tt.mean(axis=1))
#tt_norm=tt/tt.mean(axis=1)
tt_sum = tt.sum(axis=1)
print(tt.sum(axis=1))
print(np.tile(tt_sum, (3, 1)).T)
# divide each element in the row with the sum of the row so the sum will be equal to 1
tt_norm=tt/np.tile(tt_sum, (3, 1)).T
print(tt_norm)

[12 14 30]
[[12 12 12]
 [14 14 14]
 [30 30 30]]
[[0.16666667 0.33333333 0.5       ]
 [0.07142857 0.35714286 0.57142857]
 [0.66666667 0.13333333 0.2       ]]


In [101]:
def Normalize_Array(arr_to_normalize):
    arr_sum_per_row = arr_to_normalize.sum(axis=1)
    arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T
    return arr_norm

In [53]:
#edge_index = df_movies.query("movieId == 'A141HP4LYPW' and userId == 'B003AI2VGA'").index
edge_index = df_movies.query("movieId == 'A141HP4LYPW' and userId == 'B003AI2VGAzzzzz'").index
if len(edge_index) != 0:
    df_movies.iloc[edge_index[0], 2]
else:
    print('no edge found')
#df_movies.loc[edge_index[0], 'score']

no edge found


In [ ]:
arr_sum_per_row = arr_to_normalize.sum(axis=1)
arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T

In [228]:
# create a sample graph with edges
def create_sample_graph(lst_test):
    lst_test = [('A141HP4LYPW', 'B003AI2VGA', '3.0'),
     ('B000063W1R', 'A141HP4LYPW', '4.0'),
     ('6304286961', 'A141HP4LYPW', '5.0'),
     ('A141HP4LYPW', '5556167281', '5.0')]
    print(lst_test)

    df_movies = amp.Reorganize_Edges_DataFrame(lst_test)
    
    return df_movies

In [249]:
def define_P_R(df_movies):
    node_movies = df_movies['movieId'].unique()
    node_users = df_movies['userId'].unique()
    nodes = np.concatenate((node_users, node_movies), axis=0)
    print('nodes = {}'.format(nodes))
    n_nodes = len(nodes)
    P = np.zeros((n_nodes, n_nodes))

    i = 0
    for n_r in nodes:
        j = 0
        for n_c in nodes:
            edge_index = df_movies.query("movieId == '" + n_r + "' and userId == '" + n_c + "'").index
            if len(edge_index) != 0:
                P[i][j] = df_movies.iloc[edge_index[0], 2]
                P[j][i] = P[i][j]
            j += 1
        i += 1
        
    P_norm = Normalize_Array(P)
    R = np.diag(np.ones(n_nodes), 0)
    beta = 0.15
    #R *= beta
    R_zero = R.copy()
        
    return P_norm, R, R_zero

In [250]:
# Preparation and random walk
df_movies = create_sample_graph(lst_test)
P, R, R_zero = define_P_R(df_movies)
print(R)
print(P)
#savetxt(amp.FileNameUnique(prefix = "P_norm_", suffix = '.csv'), P_norm, delimiter=',')

[('A141HP4LYPW', 'B003AI2VGA', '3.0'), ('B000063W1R', 'A141HP4LYPW', '4.0'), ('6304286961', 'A141HP4LYPW', '5.0'), ('A141HP4LYPW', '5556167281', '5.0')]
Graph has been reorganized as a DataFrame, row format: ['userId', 'movieId', 'score']
nodes = ['B003AI2VGA' 'B000063W1R' '6304286961' '5556167281' 'A141HP4LYPW']
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[[0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         1.        ]
 [0.         0.         0.         0.         1.        ]
 [0.17647059 0.23529412 0.29411765 0.29411765 0.        ]]


In [241]:
R_P = R.copy()
R_R = R.copy()

In [259]:
R_P = (1-beta)*np.dot(P_norm, R_P) + beta*R_zero
print(R_P)

# since there is no direction, using P or R as 
R_R = (1-beta)*np.dot(R_R, P_norm) + beta*R_zero
print(R_R)

[[0.23488171 0.11317561 0.14146951 0.14146951 0.36900365]
 [0.08488171 0.26317561 0.14146951 0.14146951 0.36900365]
 [0.08488171 0.11317561 0.29146951 0.14146951 0.36900365]
 [0.08488171 0.11317561 0.14146951 0.29146951 0.36900365]
 [0.06511829 0.08682439 0.10853049 0.10853049 0.63099635]]
[[0.23488171 0.11317561 0.14146951 0.14146951 0.36900365]
 [0.08488171 0.26317561 0.14146951 0.14146951 0.36900365]
 [0.08488171 0.11317561 0.29146951 0.14146951 0.36900365]
 [0.08488171 0.11317561 0.14146951 0.29146951 0.36900365]
 [0.06511829 0.08682439 0.10853049 0.10853049 0.63099635]]
